<a href="https://colab.research.google.com/github/dkurbatovv/Python/blob/main/Drone_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!mkdir -p ~/.kaggle/

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d sonainjamil/malicious-drones

In [ ]:
! unzip malicious-drones

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import glob as gb

import os
import cv2
from sklearn import preprocessing
from pathlib import Path

In [ ]:
def load_data():

  images = []
  labels = []

  dataset = "/content/UAV_Dataset/"
  label_names = ['Aeroplane', 'Bird', 'Drone', 'Helicopter', 'Malicious UAV']
  for folder in os.listdir(dataset):
    files = gb.glob(pathname=str(dataset+folder+"/*.png"))

    label = label_names.index(folder)
    for file in files:
      image = cv2.imread(file)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

      image = cv2.resize(image, (150,150))

      images.append(image)
      labels.append(label)

  images = np.array(images)
  labels = np.array(labels) 


  return images, labels   

In [ ]:
images, labels = load_data()

In [ ]:
images.shape

In [ ]:
unique, counts = np.unique(labels, return_counts = True)

In [ ]:
counts

In [ ]:
import seaborn as sns
label_names = ['Aeroplane', 'Bird', 'Drone', 'Helicopter', 'Malicious UAV']
sns.barplot(x = label_names, y = counts)

In [ ]:
labels = np.array(pd.get_dummies(labels))

In [ ]:
labels

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_Test, y_train, y_Test = train_test_split(images, labels, test_size = 0.3, random_state = 100)


In [ ]:
plt.imshow(X_train[100])
plt.title('Its a'+str(y_train[100]))

In [ ]:
model = Sequential()

#model.add(rescile)

model.add(Conv2D(filters = 32, kernel_size = (2,2), input_shape = (150,150,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation = 'softmax'))



In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs = 50, validation_split = 0.2)

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(0,50)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc_train = history.history['accuracy']
acc_val = history.history['val_accuracy']
epochs = range(0,50)
plt.plot(epochs, acc_train, 'g', label='Training Accuracy')
plt.plot(epochs, acc_val, 'b', label='validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()